In [2]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()
import tableauserverclient as TSC


my_server_url = 'https://10ax.online.tableau.com/'
my_token_name = os.getenv("my_token_name")
my_token_secret = os.getenv("my_token_secret")
my_site_id = os.getenv("my_site_id")

tableau_auth = TSC.PersonalAccessTokenAuth( token_name=my_token_name
                                           ,personal_access_token=my_token_secret
                                           ,site_id=my_site_id)
server = TSC.Server(my_server_url, use_server_version=True)

with server.auth.sign_in_with_personal_access_token(tableau_auth):
    print('[Logged in successfully to {}]'.format(my_server_url))

[Logged in successfully to https://10ax.online.tableau.com/]


In [7]:
import tableauserverclient as TSC

my_server_url = 'https://10ax.online.tableau.com/'
my_token_name = os.getenv("my_token_name")
my_token_secret = os.getenv("my_token_secret")
my_site_id = os.getenv("my_site_id")
my_workbook = "Let's Play A Game"

tableau_auth = TSC.PersonalAccessTokenAuth( token_name=my_token_name
                                           ,personal_access_token=my_token_secret
                                           ,site_id=my_site_id)
server = TSC.Server(my_server_url, use_server_version=True)

with server.auth.sign_in_with_personal_access_token(tableau_auth):
    print('[Logged in successfully to {}]'.format(my_server_url))
    
    req_option = TSC.RequestOptions()
    req_option.filter.add(TSC.Filter(TSC.RequestOptions.Field.Name
                                    ,TSC.RequestOptions.Operator.Equals
                                    ,my_workbook))
 
    workbooks, pagination_item = server.workbooks.get(req_options=req_option)
    if workbooks:
        print(f"=== Workbook ===")
        print(f"Name:{workbooks[0].name}")
        print(f"luid:{workbooks[0].id}")
        print(f"URL:{workbooks[0].webpage_url}")
        print(f"Content URL:{workbooks[0].content_url}")

[Logged in successfully to https://10ax.online.tableau.com/]
=== Workbook ===
Name:Let's Play A Game
luid:3f3869fd-7c0f-4c53-aa65-a1133e5c8a52
URL:https://10ax.online.tableau.com/#/site/hotelab/workbooks/941424
Content URL:LetsPlayAGame


In [36]:
pip install tableauserverclient pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 11.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [61]:
import os
import io
import PyPDF2  # ✅ PyPDF 사용
import tableauserverclient as TSC

# Tableau Server 정보
my_server_url = 'https://10ax.online.tableau.com/'
my_token_name = os.getenv("my_token_name")
my_token_secret = os.getenv("my_token_secret")
my_site_id = os.getenv("my_site_id")
my_workbook = "Let's Play A Game"

# 'SelectedPublisher' 매개변수 목록
publishers = [
    "Activision",
    "Electronic Arts",
    "Konami Digital Entertainment",
    "Namco Bandai Games",
    "Nintendo",
    "Sega",
    "Sony Computer Entertainment",
    "Take-Two Interactive",
    "THQ",
    "Ubisoft"
]

# Tableau 인증
tableau_auth = TSC.PersonalAccessTokenAuth(
    token_name=my_token_name,
    personal_access_token=my_token_secret,
    site_id=my_site_id
)
server = TSC.Server(my_server_url, use_server_version=True)

# PDF 저장 폴더
pdf_folder = "output"
os.makedirs(pdf_folder, exist_ok=True)

# 병합된 PDF 파일 경로
merged_pdf_path = os.path.join(pdf_folder, "merged_publisher_views.pdf")
merged_pdf_writer = PyPDF2.PdfWriter()  # ✅ PyPDF2 PdfWriter 사용

# Tableau 로그인
with server.auth.sign_in_with_personal_access_token(tableau_auth):
    print(f'[Logged in successfully to {my_server_url}]')

    # Workbook 가져오기
    req_option = TSC.RequestOptions()
    req_option.filter.add(
        TSC.Filter(
            TSC.RequestOptions.Field.Name,
            TSC.RequestOptions.Operator.Equals,
            my_workbook
        )
    )

    workbooks, pagination_item = server.workbooks.get(req_options=req_option)

    if workbooks:
        workbook = workbooks[0]
        print(f"Workbook found: {workbook.name} (ID: {workbook.id})")

        # Workbook 내 View 가져오기
        server.workbooks.populate_views(workbook)
        view = workbook.views[0]  # 첫 번째 View 사용
        print(f"Using view: {view.name} (ID: {view.id})")

        for publisher in publishers:
            print(f"Generating PDF for: {publisher}")

            # 🆕 새로운 view 객체 가져오기
            selected_view = server.views.get_by_id(view.id)

            # 🆕 필터 적용 후 PDF 변환
            pdf_req_option = TSC.PDFRequestOptions()
            pdf_req_option.vf("SelectedPublisher", publisher)  # ✅ 필터 적용

            server.views.populate_pdf(selected_view, pdf_req_option)

            # ✅ selected_view.pdf를 BytesIO 객체로 변환
            pdf_stream = io.BytesIO(selected_view.pdf)

            # ✅ PyPDF2로 PDF 읽기
            temp_pdf_reader = PyPDF2.PdfReader(pdf_stream)

            # ✅ 기존 PDF 페이지를 그대로 추가 (회전 없음)
            for page in temp_pdf_reader.pages:
                merged_pdf_writer.add_page(page)

        print("All PDFs merged successfully.")

        # 병합된 PDF 저장
        with open(merged_pdf_path, "wb") as output_pdf:
            merged_pdf_writer.write(output_pdf)

        print(f"Final merged PDF saved: {merged_pdf_path}")

DEBUG:TSC:request method get, url: https://10ax.online.tableau.com//api/2.4/serverInfo
DEBUG:TSC:[15:26:51] Begin blocking request to https://10ax.online.tableau.com//api/2.4/serverInfo
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): 10ax.online.tableau.com:443
DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/2.4/serverInfo HTTP/11" 200 269
DEBUG:TSC:[15:26:52] Call finished
DEBUG:TSC:[15:26:52] Async request returned: received <Response [200]>
DEBUG:TSC:Response status: <Response [200]>
DEBUG:TSC:Server response from https://10ax.online.tableau.com//api/2.4/serverInfo
INFO:TSC:versions: 3.25, 2.4
DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "POST /api/3.25/auth/signin HTTP/11" 200 394
DEBUG:TSC:Response status: <Response [200]>
INFO:TSC:Signed into https://10ax.online.tableau.com/ as user with id 38f9dff8-8158-4743-8c46-f928cf405f22
INFO:TSC:Querying all workbooks on site
DEBUG:TSC:request method get, url: https://10ax.onli

[Logged in successfully to https://10ax.online.tableau.com/]


DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/workbooks?filter=name%3Aeq%3ALet%27s+Play+A+Game&pageNumber=1&pageSize=100 HTTP/11" 200 603
DEBUG:TSC:[15:26:53] Call finished
DEBUG:TSC:[15:26:53] Async request returned: received <Response [200]>
DEBUG:TSC:Response status: <Response [200]>
DEBUG:TSC:Server response from https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/workbooks
INFO:TSC:Populated views for workbook (ID: 3f3869fd-7c0f-4c53-aa65-a1133e5c8a52)
DEBUG:TSC:request method get, url: https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/workbooks/3f3869fd-7c0f-4c53-aa65-a1133e5c8a52/views
DEBUG:TSC:[15:26:53] Begin blocking request to https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/workbooks/3f3869fd-7c0f-4c53-aa65-a1133e5c8a52/views


Workbook found: Let's Play A Game (ID: 3f3869fd-7c0f-4c53-aa65-a1133e5c8a52)


DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/workbooks/3f3869fd-7c0f-4c53-aa65-a1133e5c8a52/views HTTP/11" 200 340
DEBUG:TSC:[15:26:53] Call finished
DEBUG:TSC:[15:26:53] Async request returned: received <Response [200]>
DEBUG:TSC:Response status: <Response [200]>
DEBUG:TSC:Server response from https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/workbooks/3f3869fd-7c0f-4c53-aa65-a1133e5c8a52/views
INFO:TSC:Querying single view (ID: 77ff3a78-dc22-455d-b912-a870d00ac324)
DEBUG:TSC:request method get, url: https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324
DEBUG:TSC:[15:26:53] Begin blocking request to https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324


...

DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324 HTTP/11" 200 454
DEBUG:TSC:[15:26:58] Call finished
DEBUG:TSC:[15:26:58] Async request returned: received <Response [200]>
DEBUG:TSC:Response status: <Response [200]>
DEBUG:TSC:Server response from https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324
INFO:TSC:Populated pdf for view (ID: 77ff3a78-dc22-455d-b912-a870d00ac324)
DEBUG:TSC:request method get, url: https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324/pdf
DEBUG:TSC:[15:26:58] Begin blocking request to https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324/pdf
DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/3.25/sites/0

Generating PDF for: Ubisoft


DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324 HTTP/11" 200 454
DEBUG:TSC:[15:26:58] Call finished
DEBUG:TSC:[15:26:58] Async request returned: received <Response [200]>
DEBUG:TSC:Response status: <Response [200]>
DEBUG:TSC:Server response from https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324
INFO:TSC:Populated pdf for view (ID: 77ff3a78-dc22-455d-b912-a870d00ac324)
DEBUG:TSC:request method get, url: https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324/pdf
DEBUG:TSC:[15:26:58] Begin blocking request to https://10ax.online.tableau.com//api/3.25/sites/01908fc8-731c-4227-997c-7fab8d70218c/views/77ff3a78-dc22-455d-b912-a870d00ac324/pdf
DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "GET /api/3.25/sites/0

All PDFs merged successfully.
Final merged PDF saved: output/merged_publisher_views.pdf


DEBUG:urllib3.connectionpool:https://10ax.online.tableau.com:443 "POST /api/3.25/auth/signout HTTP/11" 204 0
DEBUG:TSC:[15:26:59] Call finished
DEBUG:TSC:[15:26:59] Async request returned: received <Response [204]>
DEBUG:TSC:Response status: <Response [204]>
DEBUG:TSC:Server response from https://10ax.online.tableau.com//api/3.25/auth/signout
INFO:TSC:Signed out
